In [1]:
import pandas as pd
import numpy as np

In [2]:
#url = "https://data.open-power-system-data.org/household_data/latest/household_data_15min_singleindex.csv"
url= "household_data_15min_singleindex.csv"
df = pd.read_csv(url, sep=",")


#Las energías están en kWh 
#Los precios se tomaran en €/MWh

In [120]:
#Obtencion de los datos de precios, desde archivo, si hay tiempo mirar documentacion de la API
#url_price = "https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/export?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.12.2016+00%3A00%7CUTC%7CDAY&biddingZone.values=CTY%7C10Y1001A1001A83F!BZN%7C10Y1001A1001A82H&dateTime.timezone=UTC&dateTime.timezone_input=UTC&dataItem=ALL&timeRange=YEAR&exportType=CSV"
#https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_complete_parameter_list
#https://github.com/EnergieID/entsoe-py
prices_csv="Day-ahead Prices_201601010000-201701010000.csv"
df_prices = pd.read_csv(prices_csv, sep=",")


In [126]:
df_prices.columns[0]

'MTU (UTC)'

In [128]:
def cutendtime(timestring):
    times = str(timestring).split("-")
    return times[0]

In [135]:
df_prices['time'] = df_prices[df_prices.columns[0]].apply(cutendtime)
df_prices.head()

,MTU (UTC),Day-ahead Price [EUR/MWh],time
0,01.01.2016 00:00 - 01.01.2016 01:00,22.39,01.01.2016 00:00
1,01.01.2016 01:00 - 01.01.2016 02:00,20.59,01.01.2016 01:00
2,01.01.2016 02:00 - 01.01.2016 03:00,16.81,01.01.2016 02:00
3,01.01.2016 03:00 - 01.01.2016 04:00,17.41,01.01.2016 03:00
4,01.01.2016 04:00 - 01.01.2016 05:00,17.02,01.01.2016 04:00


In [115]:
df.tail(15)

,utc_timestamp,cet_cest_timestamp,DE_KN_industrial1_grid_import,DE_KN_industrial1_pv_1,DE_KN_industrial1_pv_2,DE_KN_industrial2_grid_import,DE_KN_industrial2_pv_1,DE_KN_industrial2_storage_charge,DE_KN_industrial2_storage_decharge,DE_KN_industrial3_area_offices,...,DE_KN_residential5_refrigerator,DE_KN_residential5_washing_machine,DE_KN_residential6_circulation_pump,DE_KN_residential6_dishwasher,DE_KN_residential6_freezer,DE_KN_residential6_grid_export,DE_KN_residential6_grid_import,DE_KN_residential6_pv,DE_KN_residential6_washing_machine,interpolated_values
63924,2017-02-09T08:15:00Z,2017-02-09T09:15:00+0100,248282.156,5910.642,7685.380,13378.744,20859.814,1077.839,646.945,13418.070,...,465.530,248.703,477.433,NaN,NaN,1537.19,-4496.415,10104.836,NaN,NaN
63925,2017-02-09T08:30:00Z,2017-02-09T09:30:00+0100,248287.766,5910.682,7685.380,13379.979,20859.955,1077.844,646.945,13418.480,...,465.536,248.703,477.449,NaN,NaN,1537.19,-4496.235,10104.880,NaN,NaN
63926,2017-02-09T08:45:00Z,2017-02-09T09:45:00+0100,248293.969,5910.755,7685.422,13381.170,20860.180,1077.848,646.945,13419.261,...,465.551,248.703,477.468,NaN,NaN,1537.19,-4496.160,10104.975,NaN,DE_KN_residential5_dishwasher
63927,2017-02-09T09:00:00Z,2017-02-09T10:00:00+0100,248299.656,5910.833,7685.473,13381.725,20860.469,1077.854,646.945,13419.681,...,465.564,248.703,477.487,NaN,NaN,1537.19,-4496.120,10105.094,NaN,DE_KN_residential5_dishwasher
63928,2017-02-09T09:15:00Z,2017-02-09T10:15:00+0100,248305.016,5910.938,7685.555,13382.129,20860.840,1077.858,646.945,13420.295,...,465.577,248.703,477.506,NaN,NaN,1537.20,-4496.080,10105.261,NaN,NaN
63929,2017-02-09T09:30:00Z,2017-02-09T10:30:00+0100,248310.781,5911.077,7685.665,13382.535,20861.375,1077.864,646.945,13420.745,...,465.588,248.703,477.525,NaN,NaN,1537.21,-4496.060,10105.506,NaN,NaN
63930,2017-02-09T09:45:00Z,2017-02-09T10:45:00+0100,248317.266,5911.228,7685.777,13383.004,20861.840,1077.868,646.945,13421.436,...,465.602,248.703,477.545,NaN,NaN,1537.21,-4496.050,10105.705,NaN,NaN
63931,2017-02-09T10:00:00Z,2017-02-09T11:00:00+0100,248323.719,5911.367,7685.905,13383.391,20862.295,1077.873,646.945,13422.120,...,465.609,248.703,477.564,NaN,NaN,1537.23,-4496.030,10105.910,NaN,NaN
63932,2017-02-09T10:15:00Z,2017-02-09T11:15:00+0100,248331.375,5911.498,7686.015,13383.844,20862.791,1077.878,646.945,13422.750,...,465.609,248.703,477.584,NaN,NaN,1537.23,-4496.030,10106.125,NaN,NaN
63933,2017-02-09T10:30:00Z,2017-02-09T11:30:00+0100,248334.016,5911.535,7686.045,13384.254,20863.350,1077.883,646.945,13423.431,...,465.609,248.703,477.603,NaN,NaN,1537.23,-4496.020,10106.355,NaN,DE_KN_residential3_dishwasher | DE_KN_resident...


In [5]:
#Definición de un dataframe con los nombres de las columnas para poder trabajarlos

col_names= pd.DataFrame(list(df),columns=['col_name'])
col_names.head()

,col_name
0,utc_timestamp
1,cet_cest_timestamp
2,DE_KN_industrial1_grid_import
3,DE_KN_industrial1_pv_1
4,DE_KN_industrial1_pv_2


In [108]:
#Funciones para extraer informacion de las columnas
#La nomneclatura que siguen las columnas separa los conceptos por _ y la casa esta en el 3er campo

def gethousenumber(col_name):
    fields = str(col_name).split("_")
    house_field = 2
    return fields[house_field][-1] if len(fields)>2 else 0

def getconcept(col_name):
    fields = str(col_name).split("_")
    fields_num = len(fields)
    fixed_fields = 3
    field = list(range(fixed_fields,fields_num))
    return ''.join([fields[num] for num in field])

In [111]:
col_names['house_number'] = col_names['col_name'].apply(gethousenumber)
col_names['concept'] = col_names['col_name'].apply(getconcept)
col_names.tail(15)

,col_name,house_number,concept
58,DE_KN_residential4_pv,4,pv
59,DE_KN_residential4_refrigerator,4,refrigerator
60,DE_KN_residential4_washing_machine,4,washingmachine
61,DE_KN_residential5_dishwasher,5,dishwasher
62,DE_KN_residential5_grid_import,5,gridimport
63,DE_KN_residential5_refrigerator,5,refrigerator
64,DE_KN_residential5_washing_machine,5,washingmachine
65,DE_KN_residential6_circulation_pump,6,circulationpump
66,DE_KN_residential6_dishwasher,6,dishwasher
67,DE_KN_residential6_freezer,6,freezer


In [113]:
col_names.groupby('concept').count()
#col_names2=col_names.assign(household=)

,col_name,house_number
concept,,
,3,3
areaoffices,1,1
arearoomchemistry,1,1
arearoomclean,1,1
arearoomprinting1,1,1
arearoomprinting2,1,1
circulationpump,3,3
compressor,1,1
coolingaggregate,1,1


In [ ]:
#Crear lista con los columnas para mantener del dataframe de origen, por casas. 

In [ ]:
#Dataframeprueba
df.columns['utc_timestamp','cet_cest_timestamp',]